# Обработка и нормализация сырых данных

Для каждого датасета в этом ноутбуке выполняется стандартный алгоритм подготовки:

1. **Импорт данных**  
   Загрузка таблиц из файлов `.csv` или `.xlsx`.

2. **Первичный анализ**  
   Просмотр структуры и типов данных с помощью `.head()` и `.info()`.

3. **Преобразование в long-формат**  
   Использование `pd.melt()` для приведения таблицы к удобному для анализа виду (если требуется).

4. **Объединение листов Excel-файлов**  
   В случаях с многолистными Excel-файлами данные объединяются через `reduce()` и `pd.merge()`.

5. **Сохранение уникальных названий регионов**  
   В конце каждого блока сохраняется `.txt`-файл со списком уникальных регионов — это нужно для дальнейшей унификации и слияния всех таблиц.


In [73]:
# импорт библиотек
import pandas as pd

# data 1

In [74]:
# Загрузка данных
df_poverty = pd.read_csv("../../data/raw/poverty_percent_by_regions_1992_2020.csv")

# Посмотрим первые строки
display(df_poverty.head())

# Посмотрим информацию о датафрейме
print(df_poverty.info())


,region,year,poverty_percent
0,Российская Федерация,1992,33.5
1,Российская Федерация,1993,31.3
2,Российская Федерация,1994,22.4
3,Российская Федерация,1995,24.8
4,Российская Федерация,1996,22.1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339 entries, 0 to 2338
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   region           2339 non-null   object 
 1   year             2339 non-null   int64  
 2   poverty_percent  2339 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 54.9+ KB
None


In [75]:
# Вывести уникальные регионы
unique_regions = df_poverty["region"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/poverty_unique_regions.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 2

In [76]:
# для применения функции к элементам последовательности по очереди
from functools import reduce

# Загрузка данных
file_path = "../../data/raw/cash_real_income_wages_2015_2020.xlsx"

# Функция для обработки каждого листа
def process_income_sheet(sheet_name: str, value_column: str) -> pd.DataFrame:
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    # Удаляем строки, где все значения по годам — пропущены (например, округа)
    df = df.dropna(subset=df.columns[1:], how='all')
    # Переводим в long-формат
    df_long = pd.melt(df, id_vars=["region"], var_name="year", value_name=value_column)
    df_long["year"] = df_long["year"].astype(int)
    return df_long

# Обработка всех 4 листов
df_cash_income = process_income_sheet("per_capita_cash_income", "income_per_capita")
df_real_income = process_income_sheet("real_incomes", "real_income")
df_nominal_wage = process_income_sheet("formal_wage_paid", "nominal_wage")
df_real_wage = process_income_sheet("real_pay", "real_wage")

# Объединение по region и year
dfs = [df_cash_income, df_real_income, df_nominal_wage, df_real_wage]
df_merged_income = reduce(lambda left, right: pd.merge(left, right, on=["region", "year"], how="outer"), dfs)

# Посмотрим первые строки
display(df_merged_income.head())

# Посмотрим информацию о датафрейме
print(df_merged_income.info())

,region,year,income_per_capita,real_income,nominal_wage,real_wage
0,Российская Федерация,2015,302541.0,96.42,34030.0,91.01
1,Центральный\n федеральный округ,2015,38832.0,96.20,41961.0,90.3
2,Белгородская область,2015,28043.0,99.30,25456.0,93.2
3,Брянская область,2015,23428.0,97.00,21679.0,89
4,Владимирская область,2015,22712.0,99.50,23877.0,91


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   region             600 non-null    object 
 1   year               600 non-null    int64  
 2   income_per_capita  576 non-null    float64
 3   real_income        574 non-null    float64
 4   nominal_wage       576 non-null    float64
 5   real_wage          576 non-null    object 
dtypes: float64(3), int64(1), object(2)
memory usage: 28.2+ KB
None


In [77]:
# Вывести уникальные регионы
unique_regions = df_merged_income["region"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/cash_income_unique_regions.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 3

In [78]:
# Загрузка данных
df_workers = pd.read_csv('../../data/raw/workers.csv',sep='\t')
df_workers
# NO DATA!

,11242000300190200001 Отношение числа занятых в экономике региона к численности населения региона в трудоспособном возрасте;;;;;;;;;;
0,ЕДИНИЦА ИЗМЕРЕНИЯ;Процент;;;;;;;;;
1,ТИП СБОРА;За период;;;;;;;;;
2,"ПЕРИОД;2014 г., 2012 г., 2015 г., 2016 г., 201..."
3,"ОКАТО;643 Российская Федерация, 030 Центральны..."


# data 4

In [79]:
# Загрузка данных
df_population = pd.read_excel("../../data/raw/population.xlsx", sheet_name="Отчет")
df_population = df_population.drop(columns=df_population.columns[1])  # удалим колонку с кодами

# Удалим первые 2 строки (метаданные)
df_population = df_population.iloc[2:].reset_index(drop=True)

# Переименуем первую колонку и оставим только 2015–2020 года
df_population.columns = ["region_raw"] + [f"{2014 + i}" for i in range(len(df_population.columns) - 1)]

# Оставим только нужные годы
df_population = df_population[["region_raw", "2015", "2016", "2017", "2018", "2019", "2020"]]

# Удалим строку с названиями месяцев
df_population = df_population.iloc[1:].reset_index(drop=True)

# Переведём в long-формат
df_population_clean = df_population.melt(id_vars="region_raw", var_name="year", value_name="population")
df_population_clean["year"] = df_population_clean["year"].astype(int)
df_population_clean["population"] = pd.to_numeric(df_population_clean["population"], errors="coerce")

# Покажем результат
df_population_clean.head()

# Создадим список для будущих строк
clean_rows = []

# Идём по строкам парами: текущая и следующая
for i in range(len(df_population_clean) - 1):
    region = df_population_clean.iloc[i]["region_raw"]
    population = df_population_clean.iloc[i + 1]["population"]
    year = df_population_clean.iloc[i + 1]["year"]

    # Если текущая строка — это название (population NaN), а следующая — число
    if pd.isna(df_population_clean.iloc[i]["population"]) and pd.notna(population):
        clean_rows.append({"region": region, "year": year, "population": population})

# Собираем финальный датафрейм
df_population_fixed = pd.DataFrame(clean_rows)

# Посмотрим первые строки
display(df_population_fixed.head())

# Посмотрим информацию о датафрейме
print(df_population_fixed.info())


,region,year,population
0,Центральный федеральный округ,2015,38227656.0
1,Белгородская область,2015,1501699.0
2,Брянская область,2015,1423178.0
3,Владимирская область,2015,1575507.0
4,Воронежская область,2015,2441337.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      568 non-null    object 
 1   year        568 non-null    int64  
 2   population  568 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 13.4+ KB
None


In [80]:
# Вывести уникальные регионы
unique_regions = df_population_fixed["region"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/population_regions.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 5

In [81]:
# Загрузка данных
file_path = "../../data/raw/gross_regional_product_1996_2020.xls"

# Загрузим Excel-файл с использованием движка xlrd
xls = pd.ExcelFile(file_path, engine="xlrd")

# Загрузим первый лист (или нужный по имени)
df = pd.read_excel(xls, sheet_name=0,header=None)# Загрузка CSV, пропуская первую строку и используя вторую строку как заголовки

# Установим вторую строку (index=1) как заголовки
df.columns = df.iloc[2]

# Удалим первые две строки (index 0 и 1)
df = df.drop(index=[0, 1]).reset_index(drop=True)

# Переименуем первый столбец (в нём регионы)
df = df.rename(columns={df.columns[0]: "Регион"})

# Применим melt
df_gross = df.melt(id_vars="Регион", var_name="Год", value_name="ВРП_на_душу")

# Удалим строки с пустыми регионами или полностью пустыми значениями
df_gross = df_gross.dropna(subset=["Регион", "ВРП_на_душу"])

# Удалим округа, если они есть
df_gross = df_gross[~df_gross["Регион"].str.contains("федеральный округ", case=False)]

# Год — в int, ВРП — в float (если ещё не сделано)
df_gross["Год"] = df_gross["Год"].astype(int)
df_gross["ВРП_на_душу"] = df_gross["ВРП_на_душу"].astype(float)

# Посмотрим первые строки
display(df_gross.head())

# Посмотрим информацию о датафрейме
print(df_gross.info())

,Регион,Год,ВРП_на_душу
1,Российская Федерация,1996,12225.0
3,Белгородская область,1996,9575.6
4,Брянская область,1996,7275.3
5,Владимирская область,1996,7620.7
6,Воронежская область,1996,7651.9


<class 'pandas.core.frame.DataFrame'>
Index: 2186 entries, 1 to 2548
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Регион       2186 non-null   object 
 1   Год          2186 non-null   int64  
 2   ВРП_на_душу  2186 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 68.3+ KB
None


In [82]:
# Вывести уникальные регионы
unique_regions = df_gross["Регион"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/gross_regions.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 6 

In [83]:
# Загрузка данных
df = pd.read_excel('../../data/raw/retail_turnover_per_capita_2000_2021.xls',header=None)

# Используем вторую строку (index=1) как заголовки
new_columns = df.iloc[2].tolist()
new_columns[0] = "Регион"
df.columns = new_columns

# Удаляем строки: первую (index=0) и третью (index=2)
df = df.drop(index=[0, 1, 2]).reset_index(drop=True)

# Удаляем второй столбец (с единицами измерения)
df = df.drop(columns=df.columns[1])

# Преобразуем в long-формат
df_retail = df.melt(id_vars="Регион", var_name="Год", value_name="Оборот_на_душу")

# Очистка числовых значений
df_retail["Оборот_на_душу"] = (
    df_retail["Оборот_на_душу"]
    .astype(str)
    .str.replace("\xa0", "", regex=False)  # удаляем неразрывные пробелы
    .str.replace(",", ".", regex=False)    # заменяем запятую на точку
)

# Удаляем строки с нечисловыми значениями
df_retail = df_retail[df_retail["Оборот_на_душу"].str.match(r"^\d+(\.\d+)?$")]

# Преобразуем к нужным типам
df_retail["Оборот_на_душу"] = df_retail["Оборот_на_душу"].astype(float)
df_retail["Год"] = df_retail["Год"].astype(int)

# Посмотрим первые строки
display(df_retail.head())

# Посмотрим информацию о датафрейме
print(df_retail.info())

,Регион,Год,Оборот_на_душу
1,Российская Федерация,2000,16046.0
2,Центральный федеральный округ,2000,26062.0
3,Белгородская область,2000,11820.0
4,Брянская область,2000,8267.0
5,Владимирская область,2000,7442.0


<class 'pandas.core.frame.DataFrame'>
Index: 2142 entries, 1 to 2396
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Регион          2142 non-null   object 
 1   Год             2142 non-null   int64  
 2   Оборот_на_душу  2142 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 66.9+ KB
None


In [84]:
# Вывести уникальные регионы
unique_regions = df_retail["Регион"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/retail_turnover_per_capita_regions.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 7

In [85]:
# Загрузка данных
df = pd.read_excel('../../data/raw/child_mortality_urban_1990_2021.xls', header=None)

# Используем вторую строку (index=1) как заголовки
new_columns = df.iloc[2].tolist()
new_columns[0] = "Регион"
df.columns = new_columns

# Удаляем строки: первую (index=0) и третью (index=2)
df = df.drop(index=[0, 1, 2]).reset_index(drop=True)

# Удаляем второй столбец (с единицами измерения)
df = df.drop(columns=df.columns[1:3])

# Преобразуем в long-формат
df_child_urban = df.melt(id_vars="Регион", var_name="Год", value_name="Младенческая_смертность")

# Очистка числовых значений
df_child_urban["Младенческая_смертность"] = (
    df_child_urban["Младенческая_смертность"]
    .astype(str)
    .str.replace("\xa0", "", regex=False)  # удаляем неразрывные пробелы
    .str.replace(",", ".", regex=False)    # заменяем запятую на точку
)

# Удаляем строки с нечисловыми значениями
df_child_urban = df_child_urban[df_child_urban["Младенческая_смертность"].str.match(r"^\d+(\.\d+)?$")]

# Преобразуем к нужным типам
df_child_urban["Младенческая_смертность"] = df_child_urban["Младенческая_смертность"].astype(float)
df_child_urban["Год"] = df_child_urban["Год"].astype(int)

# Посмотрим первые строки
display(df_child_urban.head())

# Посмотрим информацию о датафрейме
print(df_child_urban.info())


,Регион,Год,Младенческая_смертность
0,Российская Федерация,1990,23902.0
1,Центральный федеральный округ,1990,5317.0
2,Белгородская область,1990,209.0
3,Брянская область,1990,198.0
4,Владимирская область,1990,221.0


<class 'pandas.core.frame.DataFrame'>
Index: 3176 entries, 0 to 3794
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Регион                   3176 non-null   object 
 1   Год                      3176 non-null   int64  
 2   Младенческая_смертность  3176 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 99.2+ KB
None


In [86]:
# Вывести уникальные регионы
unique_regions = df_child_urban["Регион"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/child_mortality_urban_regions.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 8

In [87]:
# Загрузка данных
df = pd.read_excel('../../data/raw/child_mortality_rural_1990_2021.xls', header=None)

# Используем вторую строку (index=1) как заголовки
new_columns = df.iloc[2].tolist()
new_columns[0] = "Регион"
df.columns = new_columns

# Удаляем строки: первую (index=0) и третью (index=2)
df = df.drop(index=[0, 1, 2]).reset_index(drop=True)

# Удаляем второй столбец (с единицами измерения)
df = df.drop(columns=df.columns[1:3])

# Преобразуем в long-формат
df_child_rural = df.melt(id_vars="Регион", var_name="Год", value_name="Младенческая_смертность")

# Очистка числовых значений
df_child_rural["Младенческая_смертность"] = (
    df_child_rural["Младенческая_смертность"]
    .astype(str)
    .str.replace("\xa0", "", regex=False)  # удаляем неразрывные пробелы
    .str.replace(",", ".", regex=False)    # заменяем запятую на точку
)

# Удаляем строки с нечисловыми значениями
df_child_rural = df_child_rural[df_child_rural["Младенческая_смертность"].str.match(r"^\d+(\.\d+)?$")]

# Преобразуем к нужным типам
df_child_rural["Младенческая_смертность"] = df_child_rural["Младенческая_смертность"].astype(float)
df_child_rural["Год"] = df_child_rural["Год"].astype(int)

# Посмотрим первые строки
display(df_child_rural.head())

# Посмотрим информацию о датафрейме
print(df_child_rural.info())

,Регион,Год,Младенческая_смертность
0,Российская Федерация,1990,11186.0
1,Центральный федеральный округ,1990,1615.0
2,Белгородская область,1990,103.0
3,Брянская область,1990,124.0
4,Владимирская область,1990,80.0


<class 'pandas.core.frame.DataFrame'>
Index: 3159 entries, 0 to 3794
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Регион                   3159 non-null   object 
 1   Год                      3159 non-null   int64  
 2   Младенческая_смертность  3159 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 98.7+ KB
None


In [88]:
# Вывести уникальные регионы
unique_regions = df_child_rural["Регион"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/child_mortality_rural_regions.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 9

In [89]:
# Загрузка данных
df_disabled = pd.read_csv('../../data/raw/disabled_total_by_age_2017_2022.csv')

# Удаляем строки с пропущенными значениями
df_disabled = df_disabled.dropna()

# Посмотрим первые строки
display(df_disabled.head())

# Посмотрим информацию о датафрейме
print(df_disabled.info())

,region,total,18_30,31_40,41_50,51_60,60_,date
0,Российская Федерация,11640873.0,550895.0,766054.0,1030652.0,2135436.0,7157836.0,2017-01-01
1,Центральный федеральный округ,3420310.0,118579.0,172662.0,257484.0,598102.0,2273483.0,2017-01-01
2,Белгородская область,223030.0,6318.0,10383.0,16596.0,37444.0,152289.0,2017-01-01
3,Брянская область,110418.0,4215.0,6568.0,10230.0,21481.0,67924.0,2017-01-01
4,Владимирская область,133352.0,4454.0,6811.0,9606.0,23322.0,89159.0,2017-01-01


<class 'pandas.core.frame.DataFrame'>
Index: 6073 entries, 0 to 6079
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   region  6073 non-null   object 
 1   total   6073 non-null   float64
 2   18_30   6073 non-null   float64
 3   31_40   6073 non-null   float64
 4   41_50   6073 non-null   float64
 5   51_60   6073 non-null   float64
 6   60_     6073 non-null   float64
 7   date    6073 non-null   object 
dtypes: float64(6), object(2)
memory usage: 427.0+ KB
None


In [90]:
# Вывести уникальные регионы
unique_regions = df_disabled["region"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/disabled_total_by_age_regions.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 10

In [91]:
# Загрузка данных
df = pd.read_excel('../../data/raw/morbidity_2005_2020_age_disease.xls', header=None)

# Сохраняем строку с названиями (года и первые 3 колонки)
new_columns = df.iloc[2].copy()
new_columns.iloc[0:3] = ["region", "disease", "age"]

# Создаём очищенную таблицу, начиная с строки 3
df_clean = df.iloc[3:].copy()
df_clean.columns = new_columns

# Удаляем строки с пропущенными region/disease
df_clean = df_clean.dropna(subset=["region", "disease"])

# Приводим числовые значения к float
for col in df_clean.columns[3:]:
    df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")

# Удаляем строки, где во всех столбцах, начиная с 4-го, пропущены значения
df_clean = df_clean.dropna(how='all', subset=df_clean.columns[3:])   

# Преобразуем таблицу из широкого формата в длинный (long format)
df_morbidity = pd.melt(
    df_clean,
    id_vars=["region", "disease", "age"],
    var_name="year", 
    value_name="cases"                     
)

# Преобразуем год в целое число
df_morbidity["year"] = df_morbidity["year"].astype(int)

# Удаляем строки с пропущенными значениями в столбце "cases"
df_morbidity.dropna(subset=["cases"], inplace=True)

# Посмотрим первые строки
display(df_morbidity.head())

# Посмотрим информацию о датафрейме
print(df_morbidity.info())

,region,disease,age,year,cases
0,Российская Федерация,"Беременность, роды и послеродовой период",0-14 лет,2005,21.3
1,Российская Федерация,"Беременность, роды и послеродовой период",15-17 лет,2005,1537.3
2,Российская Федерация,"Беременность, роды и послеродовой период",18 лет и старше,2005,6731.7
3,Российская Федерация,"Беременность, роды и послеродовой период",Всего,2005,5719.4
4,Российская Федерация,Болезни глаза и его придаточного аппарата,0-14 лет,2005,5643.4


<class 'pandas.core.frame.DataFrame'>
Index: 75451 entries, 0 to 97839
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   region   75451 non-null  object 
 1   disease  75451 non-null  object 
 2   age      75451 non-null  object 
 3   year     75451 non-null  int64  
 4   cases    75451 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 3.5+ MB
None


In [92]:
# Вывести уникальные регионы
unique_regions = df_morbidity["region"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/morbidity_2005_2020_age_disease_regions.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 11

In [93]:

# Загрузка файла
df_welfare = pd.read_excel("../../data/raw/welfare_expense_share_2015_2020.xlsx", sheet_name="data")

# Удалим строки, где все значения по годам — NaN (это округа)
df_welfare_cleaned = df_welfare.dropna(subset=[2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0], how='all')

# Переводим wide → long формат
df_welfare_long = pd.melt(df_welfare_cleaned,
                          id_vars=["region"],
                          var_name="year",
                          value_name="welfare_percent")

# Преобразуем year к int
df_welfare_long["year"] = df_welfare_long["year"].astype(int)

# Посмотрим первые строки
display(df_welfare_long.head())

# Посмотрим информацию о датафрейме
print(df_welfare_long.info())

,region,year,welfare_percent
0,Российская Федерация,2015,15.8
1,Белгородская область,2015,11.3
2,Брянская область,2015,22.0
3,Владимирская область,2015,18.1
4,Воронежская область,2015,15.2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 516 entries, 0 to 515
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   region           516 non-null    object 
 1   year             516 non-null    int64  
 2   welfare_percent  516 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 12.2+ KB
None


In [94]:
# Вывести уникальные регионы
unique_regions = df_welfare_long["region"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/welfare_expense_share_2015_2020_regions.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 12

In [95]:
# Загрузка данных
df1 = pd.read_csv("../../data/raw/regional_production_2005_2016.csv")
df2 = pd.read_csv("../../data/raw/regional_production_2017_2020.csv")

# Перевод в long
df1_long = pd.melt(df1, id_vars=["region", "production_field"],
                   var_name="year", value_name="value")
df2_long = pd.melt(df2, id_vars=["region", "production_field"],
                   var_name="year", value_name="value")

# Объединение
df_regional_production = pd.concat([df1_long, df2_long], ignore_index=True)

# Приведение типов и очистка
df_regional_production["year"] = df_regional_production["year"].astype(int)
df_regional_production = df_regional_production.dropna(subset=["value"])

# Посмотрим первые строки
display(df_regional_production.head())

# Посмотрим информацию о датафрейме
print(df_regional_production.info())

,region,production_field,year,value
0,Российская Федерация,РАЗДЕЛ С ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,2005,3.062460e+09
1,Российская Федерация,Подраздел СА ДОБЫЧА ТОПЛИВНО-ЭНЕРГЕТИЧЕСКИ...,2005,2.686256e+09
2,Российская Федерация,"Добыча каменного угля,бурого угля и торфа",2005,2.321794e+08
3,Российская Федерация,Добыча сырой нефти и природного газа; ...,2005,2.450541e+09
4,Российская Федерация,Добыча урановой и ториевой руд,2005,3.535251e+06


<class 'pandas.core.frame.DataFrame'>
Index: 11276 entries, 0 to 13563
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   region            11276 non-null  object 
 1   production_field  11276 non-null  object 
 2   year              11276 non-null  int64  
 3   value             11276 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 440.5+ KB
None


In [96]:
# Вывести уникальные регионы
unique_regions = df_regional_production["region"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/regional_production_regions.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 13

In [97]:
# для поиска файлов по шаблону в файловой системе
import glob

# Загрузка данных
socdem_files = sorted(glob.glob("../../data/raw/poverty_socdem_20*.xls"))

# Функция очистки
def clean_socdem(filepath):
    year = int(filepath.split("_")[-1].split(".")[0])
    df = pd.read_excel(filepath, header=None)
    df_clean = df.iloc[3:].copy()
    new_columns = df.iloc[2].tolist()
    new_columns[0] = "region"
    df_clean.columns = new_columns
    df_clean = df_clean.drop(columns=[new_columns[1]])  # удаляем столбец с 100
    df_clean["year"] = year
    for col in df_clean.columns[1:]:
        df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")
    return df_clean.reset_index(drop=True)

# Обработка всех файлов
df_socdem_all = pd.concat([clean_socdem(f) for f in socdem_files], ignore_index=True)

# Посмотрим первые строки
display(df_socdem_all.head())

# Посмотрим информацию о датафрейме
print(df_socdem_all.info())


,region,Дети в возрасте до 16 лет,Население старше трудоспособного возраста,Население трудоспособного возраста,year
0,Российская Федерация,39.3,6.6,54.1,2017
1,Белгородская область,43.4,11.8,44.8,2017
2,Брянская область,42.9,4.9,52.2,2017
3,Владимирская область,34.8,8.6,56.6,2017
4,Воронежская область,38.6,5.9,55.6,2017


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 5 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   region                                     352 non-null    object 
 1   Дети в возрасте до 16 лет                  340 non-null    float64
 2   Население старше трудоспособного возраста  340 non-null    float64
 3   Население трудоспособного возраста         340 non-null    float64
 4   year                                       352 non-null    int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 13.9+ KB
None


In [98]:
# Вывести уникальные регионы
unique_regions = df_socdem_all["region"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/poverty_socdem_20*_regions.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 14

In [99]:
# Загрузка данных
file_path = "../../data/raw/housing_2020.xlsx"
df_cond = pd.read_excel(file_path, sheet_name="housing_cond")
df_intent = pd.read_excel(file_path, sheet_name="housing_intent")

# Удаление строк с NaN во всех столбцах, кроме региона
df_cond = df_cond.dropna(subset=df_cond.columns[1:], how="all")
df_intent = df_intent.dropna(subset=df_intent.columns[1:], how="all")

# Переименование столбцов 
df_cond = df_cond.rename(columns={
    df_cond.columns[0]: "region",
    df_cond.columns[1]: "all_households_pct",
    df_cond.columns[2]: "not_crowded_pct",
    df_cond.columns[3]: "somewhat_crowded_pct",
    df_cond.columns[4]: "very_crowded_pct",
    df_cond.columns[5]: "no_answer_pct",
    df_cond.columns[6]: "total_area_per_person",
    df_cond.columns[7]: "living_area_per_person",
    df_cond.columns[8]: "rooms_per_household"
})

df_intent = df_intent.rename(columns={
    df_intent.columns[0]: "region",
    df_intent.columns[1]: "all_households",  
    df_intent.columns[2]: "intend_improve_housing",
    df_intent.columns[3]: "intend_due_to_crowding",
    df_intent.columns[4]: "intend_due_to_bad_condition",
    df_intent.columns[5]: "intend_due_to_both",
    df_intent.columns[6]: "plan_dolevka",
    df_intent.columns[7]: "plan_queue",
    df_intent.columns[8]: "plan_resettle",
    df_intent.columns[9]: "plan_buy_or_build",
    df_intent.columns[10]: "plan_rent",
    df_intent.columns[11]: "plan_other",
    df_intent.columns[12]: "no_answer",
    df_intent.columns[13]: "not_intending_to_improve"
})

# Удаление двух "базовых" столбцов (всегда 100%)
df_cond = df_cond.drop(columns=["all_households_pct"])
df_intent = df_intent.drop(columns=["all_households"])

# Объединение по региону
df_housing = pd.merge(df_cond, df_intent, on="region", how="outer")

# Добавляем колонку 'year'
df_housing["year"] = 2020

# Посмотрим первые строки
display(df_housing.head())

# Посмотрим информацию о датафрейме
print(df_housing.info())


,region,not_crowded_pct,somewhat_crowded_pct,very_crowded_pct,no_answer_pct,total_area_per_person,living_area_per_person,rooms_per_household,intend_improve_housing,intend_due_to_crowding,...,intend_due_to_both,plan_dolevka,plan_queue,plan_resettle,plan_buy_or_build,plan_rent,plan_other,no_answer,not_intending_to_improve,year
0,Российская Федерация,80.7,15.4,3.8,0.1,24.2,16.9,2.4,13.9,6.3,...,0.6,11.1,3.8,6.3,32.1,4,42.6,0.7,85.2,2020
1,Белгородская область,86.3,11.7,2.0,0.0,25.1,17.8,2.6,12.0,4.2,...,0.4,11.2,1.6,1.6,29.5,0,56.9,0,88.0,2020
2,Брянская область,94.7,4.6,0.6,0.0,30.5,24.1,3.2,5.7,0.8,...,0.0,…*,...,...,...,...,...,...,94.3,2020
3,Bладимирская область,81.8,15.8,2.5,0.0,24.2,16.1,2.2,9.0,3.8,...,0.0,13.8,1.7,0,38.4,1.5,44.5,0,91.0,2020
4,Bоронежская область,87.0,11.4,1.6,0.1,29.0,19.9,2.6,14.2,4.2,...,0.1,15.4,3.7,0.3,29.4,1.4,49.7,0,85.5,2020


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   region                       86 non-null     object 
 1   not_crowded_pct              86 non-null     float64
 2   somewhat_crowded_pct         86 non-null     float64
 3   very_crowded_pct             86 non-null     float64
 4   no_answer_pct                86 non-null     float64
 5   total_area_per_person        86 non-null     float64
 6   living_area_per_person       86 non-null     float64
 7   rooms_per_household          86 non-null     float64
 8   intend_improve_housing       86 non-null     float64
 9   intend_due_to_crowding       86 non-null     float64
 10  intend_due_to_bad_condition  86 non-null     float64
 11  intend_due_to_both           86 non-null     float64
 12  plan_dolevka                 86 non-null     object 
 13  plan_queue            

In [100]:
# Вывести уникальные регионы
unique_regions = df_housing["region"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/housing_regions.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 15

In [101]:
# Загрузка данных
file_path = "../../data/raw/drug_alco.xlsx"

# Функция для обработки одного листа
def process_drug_sheet(sheet_name: str, diagnosis: str) -> pd.DataFrame:
    # Читаем таблицу с заголовками
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Переводим wide в long
    df_long = pd.melt(df, id_vars=[df.columns[0]], var_name="year", value_name="value")
    
    # Переименовываем колонки
    df_long.columns = ["region", "year", "value"]
    
    # Добавляем диагноз
    df_long["diagnosis"] = diagnosis
    
    # Год в int
    df_long["year"] = df_long["year"].astype(int)
    
    # Удаляем строки без данных
    return df_long.dropna(subset=["value"])

# Обработка всех листов
alco_old = process_drug_sheet("alco", "alcohol")
alco_new = process_drug_sheet("alco1718", "alcohol")
drug_old = process_drug_sheet("drugs", "drugs")
drug_new = process_drug_sheet("drug1718", "drugs")

# Объединение
df_drug_alco = pd.concat([alco_old, alco_new, drug_old, drug_new], ignore_index=True)

# Посмотрим первые строки
display(df_drug_alco.head())

# Посмотрим информацию о датафрейме
print(df_drug_alco.info())



,region,year,value,diagnosis
0,Российская Федерация,2005,147.4,alcohol
1,Центральный федеральный округ,2005,141.5,alcohol
2,Белгородская область,2005,99.6,alcohol
3,Брянская область,2005,243.8,alcohol
4,Владимирская область,2005,178.3,alcohol


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2486 entries, 0 to 2485
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   region     2486 non-null   object
 1   year       2486 non-null   int64 
 2   value      2486 non-null   object
 3   diagnosis  2486 non-null   object
dtypes: int64(1), object(3)
memory usage: 77.8+ KB
None


In [102]:
# Вывести уникальные регионы
unique_regions = df_drug_alco["region"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/drug_alco.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

# data 16

In [103]:
# Загрузка данных
df = pd.read_csv("../../data/raw/newborn_2006_2022_monthly.csv", sep=";", encoding="utf-8")

# Удаляем ненужные столбцы
df = df.loc[:, ~df.columns.str.contains("Unnamed")]

# Преобразуем числовые значения с запятыми → точки → float
for col in df.columns[1:]:
    df[col] = df[col].astype(str).str.replace(",", ".", regex=False).astype(float)

# Переводим wide → long
df_long = pd.melt(df, id_vars=["Region"], var_name="month_year", value_name="births")

# Маппинг месяцев
month_map = {
    "январь": "01", "февраль": "02", "март": "03", "апрель": "04",
    "май": "05", "июнь": "06", "июль": "07", "август": "08",
    "сентябрь": "09", "октябрь": "10", "ноябрь": "11", "декабрь": "12"
}
df_long["month"] = df_long["month_year"].str.extract(r"^(\w+)", expand=False).map(month_map)
df_long["year"] = df_long["month_year"].str.extract(r"(\d{4})").astype(int)

# Дата из года и месяца
df_long["date"] = pd.to_datetime(df_long["year"].astype(str) + "-" + df_long["month"], errors="coerce")

# Переименование
df_long.rename(columns={"Region": "region"}, inplace=True)

# Группировка по годам
df_yearly = df_long.dropna(subset=["births", "date"])
df_yearly["year"] = df_yearly["date"].dt.year
df_births_by_year = df_yearly.groupby(["region", "year"], as_index=False)["births"].sum()

# Посмотрим первые строки
display(df_births_by_year.head())

# Посмотрим информацию о датафрейме
print(df_births_by_year.info())

/var/folders/l9/krnz8s9d3f94d8xt3bv8m5680000gn/T/ipykernel_11301/1948566235.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yearly["year"] = df_yearly["date"].dt.year


,region,year,births
0,Агинский Бурятский округ (Забайкальский край),2006,1335.0
1,Агинский Бурятский округ (Забайкальский край),2007,1531.0
2,Агинский Бурятский округ (Забайкальский край),2008,1756.0
3,Агинский Бурятский округ (Забайкальский край),2009,1738.0
4,Агинский Бурятский округ (Забайкальский край),2010,1831.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1633 entries, 0 to 1632
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   region  1633 non-null   object 
 1   year    1633 non-null   int32  
 2   births  1633 non-null   float64
dtypes: float64(1), int32(1), object(1)
memory usage: 32.0+ KB
None


In [104]:
# Вывести уникальные регионы
unique_regions = df_births_by_year["region"].unique()# Сохраняем список уникальных регионов в текстовый файл
with open("../../outputs/unique_regions_dict/newborn.txt", "w", encoding="utf-8") as f:
    for region in unique_regions:
        f.write(region + "\n")

## Унификация регионов и стандартизация таблиц

— собрать все уникальные названия регионов из .txt-файлов  
— сформировать справочник `region_mapping` для нормализации написаний  
— выделить список эталонных регионов (без None и агрегатов)  
— реализовать функцию для стандартизации названий регионов в датафреймах  
— применить функцию ко всем таблицам и сохранить результаты в *_standardized.csv


In [105]:
# Путь ко всем txt-файлам
files = glob.glob("../../outputs/unique_regions_dict/*.txt")

# Список для хранения всех названий
all_regions = []

# Проходим по всем файлам
for file_path in files:
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        # Убираем переносы строк и пробелы
        cleaned = [line.strip() for line in lines if line.strip()]
        all_regions.extend(cleaned)

# Оставляем только уникальные значения
unique_regions = sorted(set(all_regions))

# Выводим
print("Количество всех уникальных названий регионов из всех файлов:")
print(len(unique_regions))

Количество всех уникальных названий регионов из всех файлов:
188


In [106]:
# Формирование эталонного справочника регионов
# Из словаря сопоставлений region_mapping извлекаем все допустимые названия регионов
# Удаляем None (агрегаты, округа и некорректные строки)
# Получаем отсортированный список уникальных нормализованных регионов

region_mapping = {
    # Центральная Россия
    "Bладимирская область": "Владимирская область",
    "Владимирская область": "Владимирская область",
    "Bоронежская область": "Воронежская область",
    "Воронежская область": "Воронежская область",
    "Bологодская область": "Вологодская область",
    "Вологодская область": "Вологодская область",
    "Белгородская область": "Белгородская область",
    "Брянская область": "Брянская область",
    "Ивановская область": "Ивановская область",
    "Калужская область": "Калужская область",
    "Костромская область": "Костромская область",
    "Курская область": "Курская область",
    "Липецкая область": "Липецкая область",
    "Московская область": "Московская область",
    "Московская обл. в старых границах": "Московская область",
    "Орловская область": "Орловская область",
    "Рязанская область": "Рязанская область",
    "Смоленская область": "Смоленская область",
    "Тамбовская область": "Тамбовская область",
    "Тверская область": "Тверская область",
    "Тульская область": "Тульская область",
    "Ярославская область": "Ярославская область",
    
    # Северо-Запад
    "Архангельская область": "Архангельская область",
    "Архангельская область (без АО)": "Архангельская область",
    "Архангельская область (кроме Ненецкого автономного округа)": "Архангельская область",
    "Архангельская область без": "Архангельская область",
    "Архангельская область без авт. округа": "Архангельская область",
    "Архангельская область без автономного округа": "Архангельская область",
    "Архангельская обл. без данных по Ненецкому авт. окр.": "Архангельская область",
    "Вологодская область": "Вологодская область",
    "Калининградская область": "Калининградская область",
    "Ленинградская область": "Ленинградская область",
    "Мурманская область": "Мурманская область",
    "Новгородская область": "Новгородская область",
    "Псковская область": "Псковская область",
    "г. Санкт-Петербург": "Санкт-Петербург",
    "г. Севастополь": "Севастополь",
    "город Санкт-Петербург": "Санкт-Петербург",
    "город Севастополь": "Севастополь",
    
    # Южный федеральный округ и Северный Кавказ
    "Астраханская область": "Астраханская область",
    "Волгоградская область": "Волгоградская область",
    "Ростовская область": "Ростовская область",
    "Республика Адыгея": "Республика Адыгея",
    "Республика Адыгея (Адыгея)": "Республика Адыгея",
    "Республика Калмыкия": "Республика Калмыкия",
    "Краснодарский край": "Краснодарский край",
    "Ставропольский край": "Ставропольский край",
    "Кабардино-Балкарская Республика": "Кабардино-Балкарская Республика",
    "Кабардино-Балкарская": "Кабардино-Балкарская Республика",
    "Карачаево-Черкесская Республика": "Карачаево-Черкесская Республика",
    "Карачаево-Черкесская": "Карачаево-Черкесская Республика",
    "Чеченская Республика": "Чеченская Республика",
    "Республика Ингушетия": "Республика Ингушетия",
    "Республика Северная Осетия - Алания": "Республика Северная Осетия - Алания",
    "Республика Северная Осетия-Алания": "Республика Северная Осетия - Алания",
    "Осетия-Алания": "Республика Северная Осетия - Алания",
    "Республика Дагестан": "Республика Дагестан",


        # Поволжье
    "Нижегородская область": "Нижегородская область",
    "Кировская область": "Кировская область",
    "Пензенская область": "Пензенская область",
    "Самарская область": "Самарская область",
    "Саратовская область": "Саратовская область",
    "Ульяновская область": "Ульяновская область",
    "Республика Мордовия": "Республика Мордовия",
    "Республика Марий Эл": "Республика Марий Эл",
    "Чувашская Республика": "Чувашская Республика",
    "Чувашская Республика - Чувашия": "Чувашская Республика",
    "Республика Татарстан": "Республика Татарстан",
    "Республика Татарстан (Татарстан)": "Республика Татарстан",
    "Республика Башкортостан": "Республика Башкортостан",
    "Удмуртская Республика": "Удмуртская Республика",
    
    # Урал
    "Свердловская область": "Свердловская область",
    "Челябинская область": "Челябинская область",
    "Пермский край": "Пермский край",
    "Курганская область": "Курганская область",
    "Тюменская область": "Тюменская область",
    "Тюменская обл.без данных по Ханты-Мансийскому и Ямало-Ненецкому авт. окр.": "Тюменская область",
    "Тюменская область (без АО)": "Тюменская область",
    "Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)": "Тюменская область",
    "Тюменская область без": "Тюменская область",
    "Тюменская область без авт. округов": "Тюменская область",
    "Тюменская область без автономного округа": "Тюменская область",
    "Ханты-Мансийский автономный округ": "Ханты-Мансийский автономный округ",
    "Ханты-Мансийский авт. округ": "Ханты-Мансийский автономный округ",
    "Ханты-Мансийский автономный округ - Югра (Тюменская область)": "Ханты-Мансийский автономный округ",
    "Ханты-Мансийский АО": "Ханты-Мансийский автономный округ",
    "Ханты-Мансийский": "Ханты-Мансийский автономный округ",
    "Ямало-Hенецкий АО": "Ямало-Ненецкий автономный округ",
    "Ямало-Ненецкий": "Ямало-Ненецкий автономный округ",
    "Ямало-Ненецкий авт. округ": "Ямало-Ненецкий автономный округ",
    "Ямало-Ненецкий автономный округ": "Ямало-Ненецкий автономный округ",
    "Ямало-Ненецкий автономный округ (Тюменская область)": "Ямало-Ненецкий автономный округ",
    
    # Сибирь
    "Алтайский край": "Алтайский край",
    "Красноярский край": "Красноярский край",
    "Кемеровская область": "Кемеровская область",
    "Кемеровская область - Кузбасс": "Кемеровская область",
    "Новосибирская область": "Новосибирская область",
    "Омская область": "Омская область",
    "Томская область": "Томская область",
    "Иркутская область": "Иркутская область",
    "Республика Алтай": "Республика Алтай",
    "Республика Хакасия": "Республика Хакасия",
    "Республика Тыва": "Республика Тыва",
    "Бурятия": "Республика Бурятия",
    "Республика Бурятия": "Республика Бурятия",
    "Республика Саха (Якутия)": "Республика Саха (Якутия)",
    "Республика Саха(Якутия)": "Республика Саха (Якутия)",
    
    # Дальний Восток
    "Амурская область": "Амурская область",
    "Камчатский край": "Камчатский край",
    "Магаданская область": "Магаданская область",
    "Приморский край": "Приморский край",
    "Сахалинская область": "Сахалинская область",
    "Хабаровский край": "Хабаровский край",
    "Чукотский авт. округ": "Чукотский автономный округ",
    "Чукотский автономный округ": "Чукотский автономный округ",
    "Еврейская автономная область": "Еврейская автономная область",

        # Города федерального значения
    "г. Москва": "Москва",
    "г.Москва": "Москва",
    "г. Санкт-Петербург": "Санкт-Петербург",
    "город Санкт-Петербург": "Санкт-Петербург",
    "город Санкт - Петербург": "Санкт-Петербург",
    "Город Санкт-Петербург город федерального значения": "Санкт-Петербург",
    "Город Москва столица Российской Федерации город федерального значения": "Москва",
    "город Москва": "Москва",
    "г. Севастополь": "Севастополь",
    "город Севастополь": "Севастополь",
    
    # Республика Крым (если есть)
    "Республика Крым": "Республика Крым",
    
    # Строки, которые будем убирать (None)
    "Российская Федерация": None,
    "г. Байконур": None,
    "Главное медицинское управление Управления делами Президента Российской Федерации": None,
    "Раздел 1. Муниципальные образования субъектов Российской Федерации": None,
    "Республика": None,
    "Поволжский район": None,
    "Приволжский": None,
    "Приволжский федеральный округ": None,
    "Центральный федеральный округ": None,
    "Центральный район": None,
    "Центральный": None,
    "Центрально-Черноземный район": None,
    "Северо-Западный федеральный округ": None,
    "Северо-Западный": None,
    "Северо-Западный район": None,
    "Северо-Кавказский федеральный округ": None,
    "Северо-Кавказский федеральный округ (до 03.06.2014)": None,
    "Северо-Кавказский федеральный округ (с 03.06.2014)": None,
    "Северо-Кавказский район": None,
    "Северо-Кавказский": None,
    "Северный район": None,
    "Уральский федеральный округ": None,
    "Уральский район": None,
    "Уральский": None,
    "Западно-Сибирский район": None,
    "Волго-Вятский район": None,
    "Восточно-Сибирский район": None,
    "Дальневосточный федеральный округ": None,
    "Дальневосточный район": None,
    "Дальневосточный": None,
    "Крымский федеральный округ": None,
    "Сибирский федеральный округ": None,
    "Сибирский": None,
    "Южный федеральный округ": None,
    "Южный федеральный округ  (до 03.06.2014)": None,
    "Южный федеральный округ  (с 03.06.2014)": None,
    "Южный федеральный округ (по 2009 год)": None,
    "Южный федеральный округ (с 2010 года)": None,
    "Южный федеральный округ (с 28.07.2016)": None,
    "Южный федеральный округ (с 29.07.2016)": None,
    "Южный": None,
    
    # Прочие некорректные или неполные строки
    "автономного округа": None,
    "автономный округ": None,
    "автономный округ - Югра": None,
    "автономных округов": None,
    "Коми-Пермяцкий округ, входящий в состав Пермского края": None,
    "Корякский округ, входящий в состав Камчатского края": None,
    "Таймырский (Долгано-Ненецкий) автономный округ (Красноярский край)": None,
    "Эвенкийский автономный округ (Красноярский край)": None,
    "Bолгоградская область": "Волгоградская область",
    "Агинский Бурятский округ (Забайкальский край)": "Забайкальский край",
    "Город федерального значения Севастополь": "Севастополь",
    "Еврейская авт. область": "Еврейская автономная область",
    "Забайкальский край": "Забайкальский край",
    "Москва в старых границах": "Москва",
    "Ненецкий авт. округ": "Ненецкий автономный округ",
    "Ненецкий автономный округ": "Ненецкий автономный округ",
    "Ненецкий автономный округ (Архангельская область)": "Ненецкий автономный округ",
    "Оренбургская область": "Оренбургская область",
    "Республика Адыгея (Адыгея) (до 03.06.2014)": "Республика Адыгея",
    "Республика Карелия": "Республика Карелия",
    "Республика Коми": "Республика Коми",
    "Республика Северная": "Республика Северная Осетия - Алания",
    "Усть-Ордынский Бурятский округ": None,
    "Чеченская и Ингушская Республики": None,
    "федеральный округ": None,
    "федеральный округ3)": None,
}

# Извлекаем все значения
all_values = list(region_mapping.values())

# Убираем None
valid_values = [v for v in all_values if v is not None]

# Оставляем уникальные
unique_standard_regions = sorted(set(valid_values))

# Выводим
print("Количество эталонных регионов:", len(unique_standard_regions))
print(unique_standard_regions)

Количество эталонных регионов: 85
['Алтайский край', 'Амурская область', 'Архангельская область', 'Астраханская область', 'Белгородская область', 'Брянская область', 'Владимирская область', 'Волгоградская область', 'Вологодская область', 'Воронежская область', 'Еврейская автономная область', 'Забайкальский край', 'Ивановская область', 'Иркутская область', 'Кабардино-Балкарская Республика', 'Калининградская область', 'Калужская область', 'Камчатский край', 'Карачаево-Черкесская Республика', 'Кемеровская область', 'Кировская область', 'Костромская область', 'Краснодарский край', 'Красноярский край', 'Курганская область', 'Курская область', 'Ленинградская область', 'Липецкая область', 'Магаданская область', 'Москва', 'Московская область', 'Мурманская область', 'Ненецкий автономный округ', 'Нижегородская область', 'Новгородская область', 'Новосибирская область', 'Омская область', 'Оренбургская область', 'Орловская область', 'Пензенская область', 'Пермский край', 'Приморский край', 'Псковск

In [107]:
import os

def standardize_and_save_strip(
    df,
    region_col,
    mapping_dict,
    output_folder="../../outputs/standardized_datasets",
    dataset_name="dataset"
):
    """
    df: датафрейм
    region_col: колонка с регионами
    mapping_dict: словарь
    output_folder: папка для сохранения
    dataset_name: имя файла
    """
    print(f"\n Обработка: {dataset_name}")

    # Убираем пробелы в колонке данных
    df["_region_clean"] = df[region_col].str.strip()

    # Убираем пробелы в ключах словаря
    clean_mapping = {k.strip(): v for k, v in mapping_dict.items()}

    # Применяем
    df["region_standard"] = df["_region_clean"].map(clean_mapping)

    # Убираем временный столбец
    df = df.drop(columns=["_region_clean"])

    # Проверяем непривязанные
    unmatched = df.loc[df["region_standard"].isna(), region_col].unique()
    if len(unmatched) > 0:
        print("‼Несопоставленные регионы:")
        for r in unmatched:
            print("-", r)
    else:
        print("Все регионы сопоставлены")

    # Создаём папку, если нет
    os.makedirs(output_folder, exist_ok=True)

    # Сохраняем
    output_path = os.path.join(output_folder, f"{dataset_name}.csv")
    df.to_csv(output_path, index=False)
    print(f"💾 Сохранено в: {output_path}")

    return df



In [108]:
# Словарь всех таблиц с указанием нужной колонки для стандартизации регионов
datasets = {
    "child_mortality_rural_standardized": (df_child_rural, "Регион"),
    "child_mortality_urban_standardized": (df_child_urban, "Регион"),
    "disabled_standardized": (df_disabled, "region"),
    "drug_alco_standardized": (df_drug_alco, "region"),
    "gross_standardized": (df_gross, "Регион"),
    "housing_standardized": (df_housing, "region"),
    "morbidity_standardized": (df_morbidity, "region"),
    "newborns_standardized": (df_births_by_year, "region"),
    "population_standardized": (df_population_fixed, "region"),
    "poverty_socdem_standardized": (df_socdem_all, "region"),
    "poverty_standardized": (df_poverty, "region"),
    "real_income_standardized": (df_merged_income, "region"),
    "regional_production_standardized": (df_regional_production, "region"),
    "retail_standardized": (df_retail, "Регион"),
    "welfare_standardized": (df_welfare_long, "region"),
}

# 🔁 Автоматическая стандартизация всех таблиц
for name, (df, region_col) in datasets.items():
    if region_col not in df.columns:
        print(f"⚠️ Таблица '{name}' не содержит колонку '{region_col}'. Пропущена.")
        continue

    standardize_and_save_strip(
        df=df,
        region_col=region_col,
        mapping_dict=region_mapping,
        dataset_name=name
    )



 Обработка: child_mortality_rural_standardized
‼Несопоставленные регионы:
- Российская Федерация
-     Центральный федеральный округ
-     Северо-Западный федеральный округ
-     Южный федеральный округ (по 2009 год)
-     Южный федеральный округ (с 2010 года)
-     Северо-Кавказский федеральный округ
-     Приволжский федеральный округ
-             Коми-Пермяцкий округ, входящий в состав Пермского края
-     Уральский федеральный округ
-     Сибирский федеральный округ
-             Таймырский (Долгано-Ненецкий) автономный округ (Красноярский край)
-             Эвенкийский автономный округ (Красноярский край)
-             Усть-Ордынский Бурятский округ
-     Дальневосточный федеральный округ
-             Корякский округ, входящий в состав Камчатского края
- Чеченская и Ингушская Республики
- Северный район
- Северо-Западный район
- Центральный район
- Волго-Вятский район
- Центрально-Черноземный район
- Поволжский район
- Северо-Кавказский район
- Уральский район
- Западно-Сибирс

💾 Сохранено в: ../../outputs/standardized_datasets/child_mortality_rural_standardized.csv

 Обработка: child_mortality_urban_standardized
‼Несопоставленные регионы:
- Российская Федерация
-     Центральный федеральный округ
-     Северо-Западный федеральный округ
-     Южный федеральный округ (по 2009 год)
-     Южный федеральный округ (с 2010 года)
-     Северо-Кавказский федеральный округ
-     Приволжский федеральный округ
-             Коми-Пермяцкий округ, входящий в состав Пермского края
-     Уральский федеральный округ
-     Сибирский федеральный округ
-             Таймырский (Долгано-Ненецкий) автономный округ (Красноярский край)
-             Эвенкийский автономный округ (Красноярский край)
-             Усть-Ордынский Бурятский округ
-     Дальневосточный федеральный округ
-             Корякский округ, входящий в состав Камчатского края
- Чеченская и Ингушская Республики
- Северный район
- Северо-Западный район
- Центральный район
- Волго-Вятский район
- Центрально-Чернозе

## Вывод

На этом этапе выполнена полная подготовка всех таблиц для дальнейшего анализа:  
данные очищены, приведены к единому формату, нормализованы названия регионов  
и сохранены стандартизированные версии таблиц. Создана основа для объединения  
в единый мастер-датасет и последующего анализа.
